In [18]:
import os
import json
import pandas as pd
import traceback

In [26]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
key=os.getenv("OPENAI_API_KEY")
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)